In [10]:
import tensorflow as tf


class QRNN():

    def __init__(self, in_size, size, conv_size=3):
        self.kernel = None
        self.batch_size = -1
        self.conv_size = conv_size
        self.c = None
        self.h = None
        self._x = None
        if conv_size == 1 or conv_size == 2:
            return print("This case is under construction")
        else:
            self.kernel = QRNNConvolution(in_size, size, conv_size)

    def _step(self, f, z, o):
        with tf.variable_scope("fo-Pool"):
            # f,z,o is batch_size x size
            f = tf.sigmoid(f)
            z = tf.tanh(z)
            o = tf.sigmoid(o)
            self.c = tf.multiply(f, self.c) + tf.multiply(1 - f, z)
            self.h = tf.multiply(o, self.c)  # h is size vector

        return self.h

    def forward(self, x):
        length = lambda mx: int(mx.get_shape()[0])

        with tf.variable_scope("QRNN/Forward"):
            if self.c is None:
                # init context cell
                self.c = tf.zeros([length(x), self.kernel.size], dtype=tf.float32)

            if self.conv_size <= 2:
                return print("This case is under construction")
            else:
                c_f, c_z, c_o = self.kernel.conv(x)
                for i in range(length(c_f)):
                    f, z, o = c_f[i], c_z[i], c_o[i]
                    self._step(f, z, o)
        return self.h

class QRNNConvolution():

    def __init__(self, in_size, size, conv_size):
        self.in_size = in_size
        self.size = size
        self.conv_size = conv_size
        self._weight_size = self.size * 3  # z, f, o

        with tf.variable_scope("QRNN/Variable/Convolution"):
            initializer = tf.random_normal_initializer()
            self.conv_filter = tf.get_variable("conv_filter", [conv_size, in_size, self._weight_size], initializer=initializer)

    def conv(self, x):
        # !! x is batch_size x sentence_length x word_length(=channel) !!
        _weighted = tf.nn.conv1d(x, self.conv_filter, stride=1, padding="SAME", data_format="NWC")

        # _weighted is batch_size x conved_size x output_channel
        _w = tf.transpose(_weighted, [1, 0, 2])  # conved_size x  batch_size x output_channel
        _ws = tf.split(_w, num_or_size_splits=3, axis=2) # make 3(f, z, o) conved_size x  batch_size x size
        return _ws

In [13]:
import numpy as np
import tensorflow as tf

class QRNNWork():
    
    def qrnn_forward(self, X, size, n_class, batch_size, conv_size):
        in_size = int(X.get_shape()[2])

        qrnn = QRNN(in_size=in_size, size=size, conv_size=conv_size)
        hidden = qrnn.forward(X)

        with tf.name_scope("QRNN-Classifier"):
            W = tf.Variable(tf.random_normal([size, n_class]), name="W")
            b = tf.Variable(tf.random_normal([n_class]), name="b")
            output = tf.add(tf.matmul(hidden, W), b)

        return output
